In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle


In [19]:
df = pd.read_csv('../Dataset/garments_worker_productivity.csv')

In [20]:
df.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [21]:
df.describe()

,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
count,1197.000000,1197.000000,1197.000000,691.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000
mean,6.426901,0.729632,15.062172,1190.465991,4567.460317,38.210526,0.730159,0.369256,0.150376,34.609858,0.735091
std,3.463963,0.097891,10.943219,1837.455001,3348.823563,160.182643,12.709757,3.268987,0.427848,22.197687,0.174488
min,1.000000,0.070000,2.900000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.233705
25%,3.000000,0.700000,3.940000,774.500000,1440.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.650307
50%,6.000000,0.750000,15.260000,1039.000000,3960.000000,0.000000,0.000000,0.000000,0.000000,34.000000,0.773333
75%,9.000000,0.800000,24.260000,1252.500000,6960.000000,50.000000,0.000000,0.000000,0.000000,57.000000,0.850253
max,12.000000,0.800000,54.560000,23122.000000,25920.000000,3600.000000,300.000000,45.000000,2.000000,89.000000,1.120437


In [22]:
df.shape

(1197, 15)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  no_of_style_change     1197 non-null   int64  
 13  no_of_workers          1197 non-null   float64
 14  actual_productivity    1197 non-null   float64
dtypes: f

In [24]:
df.isnull().sum()

date                       0
quarter                    0
department                 0
day                        0
team                       0
targeted_productivity      0
smv                        0
wip                      506
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
actual_productivity        0
dtype: int64

In [25]:
df.drop(['wip'], axis=1, inplace=True)

In [26]:
df["date"]= pd.to_datetime(df["date"])

In [27]:
df.date

0      2015-01-01
1      2015-01-01
2      2015-01-01
3      2015-01-01
4      2015-01-01
          ...    
1192   2015-03-11
1193   2015-03-11
1194   2015-03-11
1195   2015-03-11
1196   2015-03-11
Name: date, Length: 1197, dtype: datetime64[ns]

In [28]:
df["month"] = df["date"].dt.month

In [29]:
df.drop(['date'], axis=1, inplace=True)

In [30]:
df.month

0       1
1       1
2       1
3       1
4       1
       ..
1192    3
1193    3
1194    3
1195    3
1196    3
Name: month, Length: 1197, dtype: int32

In [31]:
df['department'].value_counts()

department
sweing        691
finishing     257
finishing     249
Name: count, dtype: int64

In [32]:
df['department'] = df['department'].apply(lambda x: 'finishing' if x.strip().lower().replace(" ", "") == 'finishing' else 'sweing')

In [33]:
df['department'].value_counts()

department
sweing       691
finishing    506
Name: count, dtype: int64

In [34]:
x = df.drop(['actual_productivity'], axis=1)
y = df['actual_productivity']

In [35]:
X = x.to_numpy()

In [36]:
X

array([['Quarter1', 'sweing', 'Thursday', ..., 0, 59.0, 1],
       ['Quarter1', 'finishing', 'Thursday', ..., 0, 8.0, 1],
       ['Quarter1', 'sweing', 'Thursday', ..., 0, 30.5, 1],
       ...,
       ['Quarter2', 'finishing', 'Wednesday', ..., 0, 8.0, 3],
       ['Quarter2', 'finishing', 'Wednesday', ..., 0, 15.0, 3],
       ['Quarter2', 'finishing', 'Wednesday', ..., 0, 6.0, 3]],
      dtype=object)

In [37]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [38]:
from sklearn.preprocessing import LabelEncoder

# Encode all object (string) columns
label_encoders = {}
for col in x_train.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    x_train[col] = le.fit_transform(x_train[col])
    x_test[col] = le.transform(x_test[col])  # apply same mapping to test data
    label_encoders[col] = le

In [39]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(x_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [40]:
pred_test = model_lr.predict(x_test)

# Calculate and print evaluation metrics
print("Linear Regression Results:")
print("test_MSE:", mean_squared_error(y_test, pred_test))
print("test_MAE:", mean_absolute_error(y_test, pred_test))
print("R2_score: {}".format(r2_score(y_test, pred_test)))  

Linear Regression Results:
test_MSE: 0.020952954761875524
test_MAE: 0.10636001215549981
R2_score: 0.2913123154775591


In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Random Forest
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(x_train, y_train)
pred_test_rf = rf_model.predict(x_test)

print("Random Forest Results:")
print("test_MSE:", mean_squared_error(y_test, pred_test_rf))
print("test_MAE:", mean_absolute_error(y_test, pred_test_rf))
print("R2_score:", r2_score(y_test, pred_test_rf))
print("\n")



Random Forest Results:
test_MSE: 0.015043240212509485
test_MAE: 0.07737322366433333
R2_score: 0.49119543305099256




In [42]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

xgb_model = XGBRegressor(random_state=42, n_estimators=200)
xgb_model.fit(x_train, y_train)
pred_test_xgb = xgb_model.predict(x_test)

print("XGBoost Results:")
print("test_MSE:", mean_squared_error(y_test, pred_test_xgb))
print("test_MAE:", mean_absolute_error(y_test, pred_test_xgb))
print("R2_score:", r2_score(y_test, pred_test_xgb))

XGBoost Results:
test_MSE: 0.016820053279400934
test_MAE: 0.0842526510309615
R2_score: 0.4310986327421561


In [44]:
import joblib

# Save the XGBoost model
joblib.dump(xgb_model, "gwp.pkl")


['gwp.pkl']

In [45]:
import joblib
joblib.dump(xgb_model, "../Flask/gwp.pkl")  # saves directly into Flask folder


['../Flask/gwp.pkl']